# Imports necesarios para el procesamiento de datos

In [181]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Carga del csv a data frame y eliminacion de columnas y filas totalmente nulas

In [182]:
df_air_quality_UCI = pd.read_csv(
    '../Data/raw/AirQualityUCI.csv',
    sep= ';',
    decimal=',',
    na_values=-200
    )

df_air_quality_UCI =  df_air_quality_UCI.dropna(how='all',axis=1)
df_air_quality_UCI = df_air_quality_UCI.dropna(how='all',axis=0)

# Columnas con valores perdidos y columnas sin valores perdidos

In [183]:

print('columns with missings values:\n')
columns_with_missing_values = df_air_quality_UCI.columns[df_air_quality_UCI.isna().any()].tolist()
i = 0
for col in columns_with_missing_values:
    if i < 5 and columns_with_missing_values.index(col) < len(columns_with_missing_values)-1:
        print(col,end=' ')
        i += 1
    else:
        print(col)
        i = 0
print('\n----------------------------------------------------------\n')
print('columns witouth missing values:\n')
columns_without_any_columns = [columns for columns in df_air_quality_UCI.columns if columns not in columns_with_missing_values]
print(columns_without_any_columns)

columns with missings values:

CO(GT) PT08.S1(CO) NMHC(GT) C6H6(GT) PT08.S2(NMHC) NOx(GT)
PT08.S3(NOx) NO2(GT) PT08.S4(NO2) PT08.S5(O3) T RH
AH

----------------------------------------------------------

columns witouth missing values:

['Date', 'Time']


# Agregación de variables asociadas a la fecha y cambio del indice a DateTime para utilidades del remplazo de missing values

In [184]:
df_air_quality_UCI['DateTime'] = pd.to_datetime(df_air_quality_UCI['Date'] + ' ' + df_air_quality_UCI['Time'], format='%d/%m/%Y %H.%M.%S')

df_air_quality_UCI['dia'] = df_air_quality_UCI['DateTime'].dt.day_name()
df_air_quality_UCI['hora'] = df_air_quality_UCI['DateTime'].dt.hour
df_air_quality_UCI['mes'] = df_air_quality_UCI['DateTime'].dt.month
df_air_quality_UCI['fin_de_semana'] = df_air_quality_UCI['DateTime'].dt.dayofweek >= 5
df_air_quality_UCI['Time'] = df_air_quality_UCI['Time'].str.replace('.', ':')

df_air_quality_UCI.set_index('DateTime', inplace=True)

# Comprobar columnas nuevas y las primeras 5 filas del data frame

In [185]:
print(df_air_quality_UCI.head(5))

                           Date      Time  CO(GT)  PT08.S1(CO)  NMHC(GT)  \
DateTime                                                                   
2004-03-10 18:00:00  10/03/2004  18:00:00     2.6       1360.0     150.0   
2004-03-10 19:00:00  10/03/2004  19:00:00     2.0       1292.0     112.0   
2004-03-10 20:00:00  10/03/2004  20:00:00     2.2       1402.0      88.0   
2004-03-10 21:00:00  10/03/2004  21:00:00     2.2       1376.0      80.0   
2004-03-10 22:00:00  10/03/2004  22:00:00     1.6       1272.0      51.0   

                     C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  \
DateTime                                                                       
2004-03-10 18:00:00      11.9         1046.0    166.0        1056.0    113.0   
2004-03-10 19:00:00       9.4          955.0    103.0        1174.0     92.0   
2004-03-10 20:00:00       9.0          939.0    131.0        1140.0    114.0   
2004-03-10 21:00:00       9.2          948.0    172.0        1092.0

# Funciones de graficos y la direccion donde se guarda

In [186]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(
    style="whitegrid",
    context="notebook",
    font_scale=1.1,
    palette="viridis",
    rc={
        "axes.titlesize": 16,
        "axes.titleweight": "bold",
        "axes.labelsize": 13,
        "xtick.labelsize": 11,
        "ytick.labelsize": 11,
        "grid.color": "0.9",
        "grid.linestyle": "--",
        "grid.linewidth": 1,
        "axes.edgecolor": "0.3",
        "lines.linewidth": 2,
        "patch.edgecolor": "black",
        "figure.figsize": (10, 6),
        "figure.dpi": 150
    }
)
def _ensure_dir(base_dir, sub_folder=""):
    out_dir = os.path.join(base_dir, sub_folder)
    os.makedirs(out_dir, exist_ok=True)
    return out_dir


def histogram_df(column_name, dir_path=None):
    if column_name not in df_air_quality_UCI.columns: return

    data = df_air_quality_UCI[column_name].dropna()
    plt.figure(figsize=(10, 6))
    
    sns.histplot(data, kde=True, color='C0', bins=30, edgecolor='black', linewidth=0.6)
    
    plt.title(f'Distribución de {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    
    if dir_path:
        out_dir = _ensure_dir(dir_path, "distribuciones")
        plt.savefig(os.path.join(out_dir, f"hist_{column_name}.png"), dpi=200, bbox_inches='tight')
    plt.close()


def boxplot_df(column_name, dir_path=None):
    if column_name not in df_air_quality_UCI.columns: return
    
    data = df_air_quality_UCI[column_name].dropna()
    if data.empty: return

    plt.figure(figsize=(8, 6))
    sns.boxplot(y=data, color='lightgreen', width=0.4, linewidth=1.1, fliersize=3)
    
    plt.title(f'Boxplot de {column_name}')
    plt.ylabel(column_name)
    plt.tight_layout()
    
    if dir_path:
        out_dir = _ensure_dir(dir_path, "distribuciones")
        plt.savefig(os.path.join(out_dir, f"box_{column_name}.png"), dpi=200, bbox_inches='tight')
    plt.close()


def heatmap_df(columns, dir_path=None):

    plt.figure(figsize=(12, 10))
    
    matrix_corr = df_air_quality_UCI[columns].corr()
    
    sns.heatmap(matrix_corr, annot=True, fmt=".2f", cmap='coolwarm', 
                vmin=-1, vmax=1, linewidths=0.5)
    
    plt.title('Matriz de Correlación: Variables Físicas')

    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()
    
    if dir_path:
        _ensure_dir(dir_path) 
        filename = "heatmap_correlation_matrix.png"
        plt.savefig(os.path.join(dir_path, filename), dpi=200, bbox_inches='tight')
    plt.close()


def Scatterplot_df(x_col, y_col, dir_path=None):
    plt.figure(figsize=(10, 6))
    
    sns.scatterplot(data=df_air_quality_UCI, x=x_col, y=y_col, alpha=0.5, s=30)
    
    plt.title(f'Relación: {x_col} vs {y_col}')
    plt.xlabel(f'Referencia Real ({x_col})')
    plt.ylabel(f'Sensor ({y_col})')
    plt.tight_layout()
    
    if dir_path:
        out_dir = _ensure_dir(dir_path, "correlaciones_pares")
        clean_name = f"{x_col}_vs_{y_col}".replace("(", "").replace(")", "").replace(".", "")
        plt.savefig(os.path.join(out_dir, f"scatter_{clean_name}.png"), dpi=200, bbox_inches='tight')
    plt.close()

# Histograma y box plot de cada columna con missing
### Con esto analizar el tipo de procesamiento para los missings

**Tipos de distribuciones observadas:**
- **Log-normal (sesgo derecha):** CO(GT), NOx(GT), C6H6(GT), sensores PT08.S1, PT08.S2, PT08.S3, PT08.S5
- **Normal (Gaussiana):** T, RH, AH, NO2(GT), PT08.S4(NO2)

**La manera de tratar los missing value no es con media, mediana ni moda, es usando series de tiempo.**  
*Ejemplo:* entre 11:00 con CO(GT) = 1.2 y 13:00 con CO(GT) = 1.4, el valor de las 12:00 debe interpolarse linealmente en tiempo, no ser 2.5 (mediana).

In [187]:
for columns in columns_with_missing_values:
    histogram_df(columns,'../Data/raw/missing_data/')
    boxplot_df(columns,'../Data/raw/missing_data/')

# Porcentaje de missing values por columnas

In [188]:
missing_percentages = (df_air_quality_UCI.isna().sum() / len(df_air_quality_UCI)) * 100
df_missing_percentages = missing_percentages.to_frame(name='Missing Percentage')
df_missing_percentages.reset_index(inplace=True)
df_missing_percentages.rename(columns={'index': 'Variable'}, inplace=True)
df_missing_percentages.to_csv('../Data/raw/missing_data/missing_percentages.csv', index=False)

print(df_missing_percentages.sort_values(by='Missing Percentage', ascending=False))

if 'NMHC(GT)' in df_air_quality_UCI.columns:
    df_air_quality_UCI = df_air_quality_UCI.drop(columns=['NMHC(GT)'])
if 'NMHC(GT)' in columns_with_missing_values:
    columns_with_missing_values.remove('NMHC(GT)')

         Variable  Missing Percentage
4        NMHC(GT)           90.231912
2          CO(GT)           17.986534
9         NO2(GT)           17.548360
7         NOx(GT)           17.516298
12              T            3.911510
11    PT08.S5(O3)            3.911510
3     PT08.S1(CO)            3.911510
6   PT08.S2(NMHC)            3.911510
5        C6H6(GT)            3.911510
8    PT08.S3(NOx)            3.911510
10   PT08.S4(NO2)            3.911510
13             RH            3.911510
14             AH            3.911510
1            Time            0.000000
0            Date            0.000000
15            dia            0.000000
16           hora            0.000000
17            mes            0.000000
18  fin_de_semana            0.000000


# Trata de los missing value,usando el metodo de series de tiempo, por ello anteriormente se convirtio el indice a DateTime

In [189]:
df_air_quality_UCI[columns_with_missing_values] = df_air_quality_UCI[columns_with_missing_values].interpolate(method='time') 

df_air_quality_UCI[columns_with_missing_values] = df_air_quality_UCI[columns_with_missing_values].bfill().ffill()

# Graficos y visualizaciones de los datos, no streamlit, entender los datos y con ello escoger para stream lit
- Para el mapa de calor las columnas:  CO(GT), PT08.S1(CO), C6H6(GT), PT08.S2(NMHC), NOx(GT), PT08.S3(NOx), NO2(GT), PT08.S4(NO2), PT08.S5(O3), T, RH, AH
- Para dispersión se usan 4 pares:
    - Monoxido de Carbono (CO):
        - CO(GT) para el eje x, valor real
        - PT08.S1(CO) para el eje y, sensor
    - Hidrocarburos no metalicos (referenciamos al bencano, ya que el NMHC(GT) antes de tratar tenia 90% de datos perdidos por lo que no es factible incluso si se tratan):
        - C6H6(GT) para el eje x, valor real
        - PT08.S2(NMHC) para el eje y, sensor
    - Óxidos de Nitrógeno(NOx):
        - NOx(GT) para eje x, valor real
        - PT08.S3(NOx) para eje y, sensor
    - Dioxido de Nitrógeno(NO2):
        - NO2(GT) para eje x, valor real
        - PT08.S4(NO2) para eje y, sensor


In [190]:
columns_list = ['CO(GT)','PT08.S1(CO)', 'C6H6(GT)','PT08.S2(NMHC)','NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
columns_pars = [['CO(GT)','PT08.S1(CO)'], ['C6H6(GT)','PT08.S2(NMHC)'], ['NOx(GT)', 'PT08.S3(NOx)'], ['NO2(GT)', 'PT08.S4(NO2)']]

heatmap_df(columns_list,'../Data/raw/figures/')

for col_x, col_y in columns_pars:
    Scatterplot_df(col_x, col_y,'../Data/raw/figures/')

# Guardar nuevo CSV acorde a las observaciones

In [191]:
df_air_quality_UCI.to_csv('../Data/processed/air_quality_UCI_cleaned.csv', index=True)